In [1]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import string
import re
import random
from pathlib import Path
import spacy
from tqdm import tqdm
from spacy.training.example import Example
from spacy.tokens import DocBin

In [ ]:
spacy.prefer_gpu()

True

In [ ]:
df = pd.read_csv("train_supervised_dataset.csv")
df.set_index("id", inplace=True)
df.head()

,name,good,brand
id,,,
0,Petmax Бантик леопард с красн розой 2шт,бантик,petmax
1,87191 Бусы для елки шарики_87191,бусы,NaN
2,Футболка Piazza Italia WR011446881,футболка,piazza italia
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,заколка,NaN
4,Одежда (вес) 1500,одежда,NaN


In [ ]:
replace_nan ={
    "good": "qklent",
    "brand": "loknyr"
}
df.fillna(
    replace_nan,
    inplace=True
)

In [3]:
#тут можно еще сделать ru_to_eng(дада и такие странные примеры есть)
def eng_to_ru(word: str) -> str:
    if len(word) <= 1:
        return word
    a = ord('а')
    ru_alphabet = ''.join([chr(i) for i in range(a,a+33)])
    eng_alphabet = string.ascii_lowercase[:26]
    change = {
        "a": "а",
        "e": "е",
        "o": "о",
        "k": "к",
        "3": "з",
        "p": "р",
        "c": "с",
        "m": "м",
        "x": "х",
        "t": "т",
        "y": "у",
        "z": "з",
    }

    new_word = ""
    for i in range(len(word)):
        curr = word[i]
        if curr in eng_alphabet:
            if i - 1 < 0:
                prev_letter = "j"
            else:
                prev_letter = word[i - 1]

            if i + 1 >= len(word):
                next_letter = "j"
            else:
                next_letter = word[i + 1]

            if next_letter in ru_alphabet or prev_letter in ru_alphabet:
                curr = change.get(curr, curr)
        new_word += curr

    return new_word

def only_ones(word: str) -> str:
    if word.isdigit() or re.match(r'^-?\d+(?:\.\d+)$', word):
        return "1"
    else:
        new_word = ""
        i = 0
        while i < len(word):
            curr = word[i]
            while i < len(word) and word[i].isdigit():
                curr = "1"
                i += 1
            new_word += curr
            if curr != "1":
                i += 1

    return new_word


# to add: seperate by string.punctuations (add space between words)
#(probably i can just use some tokenizer and then make string of tokenizer output)


def remove_punct(word: str) -> str:
    return word.translate(str.maketrans("", "", string.punctuation))


def preprocess_string(word: str, empty_str: str) -> str:
    new_word = ""


    word = remove_punct(word)

    if word == "товара нет" or word == "":
        new_word = empty_str
    else:
        new_word = eng_to_ru(only_ones(word).lower())

    return new_word

In [ ]:
df.name = df.name.apply(preprocess_string, empty_str="")
df.good = df.good.apply(preprocess_string, empty_str=replace_nan["good"])
df.brand = df.brand.apply(preprocess_string, empty_str=replace_nan["brand"])

# create a set of all goods
goods_set = set(df['good'])

# filter the dataframe by removing rows where the brand is in the goods set
df = df[~df['brand'].isin(goods_set)]

len(df)

24940

In [ ]:
df.head()

,name,good,brand
id,,,
0,petmax бантик леопард с красн розой 1шт,бантик,petmax
1,1 бусы для елки шарики1,бусы,loknyr
2,футболка piazza italia wr1,футболка,piazza italia
3,1 yi1xone заколка для волос для девочки,заколка,loknyr
4,одежда вес 1,одежда,loknyr


In [ ]:
# # пройтись по goods and brands и удалить все двусмысленные ряды? (или можно оставить только 1 из многих, но выбрать 1 из многих надо
# # наверное по теории вероятности, но это опять же не бейзлайн => сделаю как-нибудь потом)
# # в бейзлайне лучше всего удалять из brands так как там чаще встречается булщит
# #тут можно было удалять только индексы и удалять их можно было бинарынм поиском
# i = 0
# while i < len(brands):
#     flag = 0
#     for j in range(len(goods)):
#         if brands[i] == goods[j]:
#             brands.pop(i)
#             flag = 1
#             break
#     print (brands[i])
#     if not flag:
#         i += 1

In [ ]:
word = ""
pattern = re.compile(r'{}'.format(word))

def find_word_indices(string, word):
    matches = pattern.finditer(string)
    for match in matches:
        if match.group(0) == word:
            start_index = match.start()
            end_index = match.end()
            return [start_index, end_index]
    return []

In [ ]:
# ner_brands = []
# ner_goods = []
train_dataset = []
for n, b, g in zip(df.name, df.brand, df.good):
    if b != replace_nan["brand"]:
        pattern = re.compile(r'{}'.format(b))
        brand_indexes = find_word_indices(n, b) + ["BRAND"]
    else:
        brand_indexes = []

    if g != replace_nan["good"]:
        pattern = re.compile(r'{}'.format(g))
        good_indexes = find_word_indices(n, g) + ["GOOD"]
    else:
        good_indexes = []

    # ner_brands.append(brand_indexes + ["BRAND"])
    # ner_goods.append(good_indexes + ["GOOD"])

    ent = []
    if brand_indexes and len(brand_indexes) == 3:
        ent.append(brand_indexes)
    if good_indexes and len(good_indexes) == 3:
        ent.append(good_indexes)
    if ent:
        train_dataset.append(
            (
                n,
                {
                    "entities": ent
                }
            )
        )


In [ ]:
# train_dataset = [
#     (n,
#      {'entities': ([b] if b[0] != -1 else []) + ([g] if g[0] != -1 else [])}
#     )
#     for n, b, g in zip(names, ner_brands, ner_goods)
# ]

# train_dateset = filter(lambda x: x[1]["entities"] != [], train_dataset)
# train_dataset

In [ ]:
n = len(train_dataset)
train, val = train_dataset[:int(n * 0.8)], train_dataset[int(n * 0.8):]

разделить пробелом: обоиавыа т.е после того как регулярка нашла слово и знает его индексы надо просто добавить пробел справа/слева если его там нет

In [ ]:
def create_training(TRAIN_DATA):
    db = DocBin()
    skipped = 0
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                #print (text)
                #print ("Skipping entity")
                skipped += 1
            else:
                ents.append(span)
        try:
            doc.ents = ents
        except ValueError:
            skipped += 1
        db.add(doc)
    print (skipped)
    return (db)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install spacy-transformers

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

2023-06-13 12:34:19.033006: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
# count = 0
# for x in brands:
#     for y in goods:
#         if x == y:
#             count += 1

некоторые слова есть в goods и в brands => нужно сделать какую-то fancy штуку, но мне в падлу(сделаю после бейзлайна) https://explosion.ai/blog/spancat

In [ ]:
# count

330

In [ ]:
nlp = spacy.blank("ru")
train_data = create_training(train)
train_data.to_disk("train_data.spacy")

100%|██████████| 19169/19169 [00:05<00:00, 3550.64it/s]


1609


In [ ]:
val_data = create_training(val)
val_data.to_disk("val.spacy")

100%|██████████| 4793/4793 [00:01<00:00, 4244.62it/s]


392


dataset is ready, now we can train a model

In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./val.spacy --gpu-id 0

2023-06-13 12:36:30.523263: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-06-13 12:36:35,098] [INFO] Set up nlp object from config
[2023-06-13 12:36:35,114] [INFO] Pipeline: ['transformer', 'ner']
[2023-06-13 12:36:35,118] [INFO] Created vocabulary
[2023-06-13 12:36:35,120] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained 

evaluating model

In [80]:
import spacy_transformers
from spacy import displacy

In [8]:
nlp = spacy.load("output\content\output\model-best")

In [ ]:
train_dataset[0][0]

'petmax бантик леопард с красн розой 1шт'

In [ ]:
sentence = train_dataset[11][0]

doc = nlp(sentence)

from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
for ent in doc.ents:
    print(ent.label_, ent.text)

GOOD колготки


In [ ]:
!zip -r /content/output.zip /content/output

  adding: content/output/ (stored 0%)
  adding: content/output/model-last/ (stored 0%)
  adding: content/output/model-last/transformer/ (stored 0%)
  adding: content/output/model-last/transformer/model (deflated 8%)
  adding: content/output/model-last/transformer/cfg (stored 0%)
  adding: content/output/model-last/meta.json (deflated 56%)
  adding: content/output/model-last/vocab/ (stored 0%)
  adding: content/output/model-last/vocab/strings.json (deflated 88%)
  adding: content/output/model-last/vocab/vectors (deflated 45%)
  adding: content/output/model-last/vocab/lookups.bin (stored 0%)
  adding: content/output/model-last/vocab/key2row (stored 0%)
  adding: content/output/model-last/vocab/vectors.cfg (stored 0%)
  adding: content/output/model-last/config.cfg (deflated 61%)
  adding: content/output/model-last/ner/ (stored 0%)
  adding: content/output/model-last/ner/model (deflated 8%)
  adding: content/output/model-last/ner/moves (deflated 53%)
  adding: content/output/model-last/ner

In [ ]:
from google.colab import files
files.download("/content/output.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [104]:
test = pd.read_csv("test_dataset.csv")
test.head()

,id,name
0,0,"469-210 ЕРМАК Клей универсальный, 15мл, блистер"
1,1,Торт СЛАДУШКА Зимняя вишня 700г
2,2,"Смеситель ""CALORIE"" 1023 А06 д/кухни"
3,3,Лимон 50гр БАР
4,4,"Коньяк САРАДЖИШВИЛИ 5 лет 0,5л Грузия"


In [106]:
#почему не обрабатывалась kонфета: потому что k была заглавной
start_name = test.name
test.name = test.name.apply(preprocess_string, empty_str="")
test.head()

,id,name
0,0,1 ермак клей универсальный 1мл блистер
1,1,торт сладушка зимняя вишня 1г
2,2,смеситель calorie 1 а1 дкухни
3,3,лимон 1гр бар
4,4,коньяк сараджишвили 1 лет 1л грузия


In [87]:
goods = []
brands = []
for x in test.name:
    doc = nlp(x)
    curr_g = []
    curr_b = []
    for ent in doc.ents:
        if ent.label_ == "GOOD":
            curr_g.append(ent.text)
        elif ent.label_ == "BRAND":
            curr_b.append(ent.text)
    goods.append(curr_g)
    brands.append(curr_b)

In [88]:
goods

[['клей'],
 ['торт'],
 ['смеситель'],
 ['лимон'],
 ['коньяк'],
 ['пластина'],
 ['рис'],
 ['труба'],
 ['консервы'],
 [],
 ['аромапалочки'],
 [],
 ['сироп'],
 [],
 [],
 ['спицы'],
 ['конфеты'],
 ['скумбрия'],
 ['чипсы'],
 [],
 ['леденцы'],
 ['печенье'],
 ['майонез'],
 ['штукатурка'],
 ['шайба'],
 ['корм'],
 ['саморез'],
 ['томаты'],
 ['саморез'],
 ['нитки'],
 ['держатель'],
 ['рюкзак'],
 ['джинсы'],
 ['консервы'],
 ['винт'],
 ['сок'],
 ['куртка'],
 ['серьги'],
 ['пельмени'],
 ['джеггинсы'],
 ['мойка'],
 ['фильтр'],
 ['авиабилеты', 'воды'],
 [],
 ['платье'],
 ['футболка'],
 [],
 ['антенна'],
 ['болт'],
 ['линолеум'],
 ['болт'],
 ['коврик'],
 ['пиво'],
 ['тархун', 'лимонад'],
 ['пенал'],
 ['ремень'],
 ['футболка'],
 ['шоколад'],
 ['молоко'],
 [],
 ['берет'],
 ['цифра'],
 ['диск отрезной'],
 ['рюкзак'],
 [],
 ['дезодорант'],
 ['болт'],
 [],
 ['туфли'],
 [],
 ['молоко'],
 ['футболка'],
 ['арахис'],
 ['пленка'],
 ['арахис'],
 ['оксигенотерапия'],
 ['лимонад'],
 ['мармелад'],
 [],
 ['опора'],


In [90]:
init_goods = goods
init_brands = brands

In [91]:
goods = list(map(lambda x: [eng_to_ru(g) for g in x], goods))
brands = list(map(lambda x: [eng_to_ru(b) for b in x], brands))

In [92]:
goods

[['клей'],
 ['торт'],
 ['смеситель'],
 ['лимон'],
 ['коньяк'],
 ['пластина'],
 ['рис'],
 ['труба'],
 ['консервы'],
 [],
 ['аромапалочки'],
 [],
 ['сироп'],
 [],
 [],
 ['спицы'],
 ['конфеты'],
 ['скумбрия'],
 ['чипсы'],
 [],
 ['леденцы'],
 ['печенье'],
 ['майонез'],
 ['штукатурка'],
 ['шайба'],
 ['корм'],
 ['саморез'],
 ['томаты'],
 ['саморез'],
 ['нитки'],
 ['держатель'],
 ['рюкзак'],
 ['джинсы'],
 ['консервы'],
 ['винт'],
 ['сок'],
 ['куртка'],
 ['серьги'],
 ['пельмени'],
 ['джеггинсы'],
 ['мойка'],
 ['фильтр'],
 ['авиабилеты', 'воды'],
 [],
 ['платье'],
 ['футболка'],
 [],
 ['антенна'],
 ['болт'],
 ['линолеум'],
 ['болт'],
 ['коврик'],
 ['пиво'],
 ['тархун', 'лимонад'],
 ['пенал'],
 ['ремень'],
 ['футболка'],
 ['шоколад'],
 ['молоко'],
 [],
 ['берет'],
 ['цифра'],
 ['диск отрезной'],
 ['рюкзак'],
 [],
 ['дезодорант'],
 ['болт'],
 [],
 ['туфли'],
 [],
 ['молоко'],
 ['футболка'],
 ['арахис'],
 ['пленка'],
 ['арахис'],
 ['оксигенотерапия'],
 ['лимонад'],
 ['мармелад'],
 [],
 ['опора'],


In [21]:
sentence = "Ifsfwlovefdsfprogrammingwith\sf$#@$python@#$@"
index = sentence.index("python")
print(sentence[index:index+len("python")])

python


чтобы в тесте все брэнды не были .lower()

In [97]:
start_name = test.name.apply(eng_to_ru).apply(remove_punct)

In [137]:
new_goods = []
new_brands = []
for name, good, brand in zip(start_name, goods, brands):
    name_lower = eng_to_ru(only_ones(remove_punct(name.lower())))
    curr_g = []
    curr_b = []
    for g in good:
        #print (name_lower, '   ', g)
        try:
            start = name_lower.index(g)
            curr_g.append(only_ones(remove_punct(name))[start:start + len(g)])
        except ValueError:
            curr_g.append(g)
    for b in brand:
        #print (name_lower, '  ', b)
        try:
            start = name_lower.index(b)
            curr_b.append(name[start:start + len(b)])
        except ValueError:
            curr_b.append(b)
    #print (curr_g)
    new_goods.append(curr_g)
    new_brands.append(curr_b)
    
    

In [138]:
new_goods

[['Клей'],
 ['Торт'],
 ['Смеситель'],
 ['Лимон'],
 ['Коньяк'],
 ['Пластина'],
 ['Рис'],
 ['Труба'],
 ['Консервы'],
 [],
 ['Аромапалочки'],
 [],
 ['СИРОП'],
 [],
 [],
 ['cпицы'],
 ['Конфеты'],
 ['Скумбрия'],
 ['Чипсы'],
 [],
 ['леденцы'],
 ['ПЕЧЕНЬЕ'],
 ['Майонез'],
 ['Штукатурка'],
 ['Шайба'],
 ['Корм'],
 ['Саморез'],
 ['ТОМАТЫ'],
 ['Саморез'],
 ['нитkи'],
 ['ДЕРЖАТЕЛЬ'],
 ['Рюкзак'],
 ['Джинсы'],
 ['консервы'],
 ['Винт'],
 ['Сок'],
 ['КУРТКА'],
 ['Серьги'],
 ['Пельмени'],
 ['Джеггинсы'],
 ['мойка'],
 ['Фильтр'],
 ['Авиабилеты', 'Воды'],
 [],
 ['Платье'],
 ['Футболка'],
 [],
 ['АНТЕННА'],
 ['Болт'],
 ['Линолеум'],
 ['Болт'],
 ['Коврик'],
 ['Пиво'],
 ['Тархун', 'Лимонад'],
 ['Пенал'],
 ['Ремень'],
 ['ФУТБОЛКА'],
 ['Шоколад'],
 ['Молоко'],
 [],
 ['БЕРЕТ'],
 ['Цифра'],
 ['Диск отрезной'],
 ['Рюкзак'],
 [],
 ['Дезодорант'],
 ['Болт'],
 [],
 ['ТУФЛИ'],
 [],
 ['Молоко'],
 ['ФУТБОЛКА'],
 ['Арахис'],
 ['Пленка'],
 ['АРАХИС'],
 ['Оксигенотерапия'],
 ['Лимонад'],
 ['мармелад'],
 [],
 ['Опора'],


In [124]:
s = "авыавы kонфеты авыа"
eng_to_ru(s)

'авыавы конфеты авыа'

In [99]:
for idx, x in enumerate(goods):
    if x == ['т', 'а', 'р', 'х', 'у', 'н', 'л', 'и', 'м', 'о', 'н', 'а', 'д']:
        displacy.render(nlp(test.name[idx]), style="ent", jupyter=True)

In [139]:
submission = pd.DataFrame(
    {
        "id": [i for i in range(len(goods))],
        "good": list(map(lambda x: ', '.join(x), new_goods)),
        "brand": list(map(lambda x: ', '.join(x), new_brands))
    }
)
submission.head()

,id,good,brand
0,0,Клей,9-210
1,1,Торт,СЛАДУШКА
2,2,Смеситель,"""CALORI"
3,3,Лимон,
4,4,Коньяк,САРАДЖИШВИЛИ


In [148]:
def remove_necessary_punctuations(word: str) -> str:
    return word.translate(str.maketrans("", "", "\"\',./!@#$%^&*()+=;[]{}"))


In [149]:
sumbmission1 = submission.apply(
    {
        "id": lambda x: x,
        "good": remove_necessary_punctuations,
        "brand": remove_necessary_punctuations,
    }
)
sumbmission1.head()

,id,good,brand
0,0,Клей,9-210
1,1,Торт,СЛАДУШКА
2,2,Смеситель,CALORI
3,3,Лимон,
4,4,Коньяк,САРАДЖИШВИЛИ


In [150]:
sumbmission1.to_csv("subm_no_punct.csv")

Посчитаем f1 на val

In [ ]:
val_data

In [ ]:
val_data = df.iloc[int(len(df) * 0.9):]
val_data.head()

,name,good,brand
id,,,
22498,черноголовка байкал стекло 1 л,qklent,черноголовка
22499,щербет кавказский 1кг,щербет,loknyr
22500,крепеж 1 мм,крепеж,loknyr
22501,ушат шайка цветы липа 1л,ушат,loknyr
22502,чаша 1л артм1,чаша,loknyr


In [ ]:
goods = []
brands = []
for x in val_data.name:
    doc = nlp(x)
    curr_g = []
    curr_b = []
    for ent in doc.ents:
        if ent.label_ == "GOOD":
            curr_g.append(ent.text)
        elif ent.label_ == "BRAND":
            curr_b.append(ent.text)
    goods.append(curr_g)
    brands.append(curr_b)

In [ ]:
val_data["good"] = val_data["good"].apply(lambda x: x if x != "qklent" else "")
val_data["brand"] = val_data["brand"].apply(lambda x: x if x != "loknyr" else "")


<ipython-input-36-8ff346071cb6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data["good"] = val_data["good"].apply(lambda x: x if x != "qklent" else "")
<ipython-input-36-8ff346071cb6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data["brand"] = val_data["brand"].apply(lambda x: x if x != "loknyr" else "")


In [ ]:
val_data.head()

,name,good,brand
id,,,
22498,черноголовка байкал стекло 1 л,,черноголовка
22499,щербет кавказский 1кг,щербет,
22500,крепеж 1 мм,крепеж,
22501,ушат шайка цветы липа 1л,ушат,
22502,чаша 1л артм1,чаша,


In [ ]:
goods = list(map(lambda x: ', '.join(x), goods)),
brands = list(map(lambda x: ', '.join(x), brands))


In [ ]:
#(F1_good + 2 * F1_brand) / 3
from sklearn.metrics import f1_score

In [ ]:
def f1_final(goods, brands):
    f1_good = f1_score(*goods, val_data.good.tolist(), average='macro')
    f1_brand = f1_score(brands, val_data.brand.tolist(), average="macro")
    f1_final = (f1_good + 2 * f1_brand) / 3
    return f1_final

In [ ]:
f1_final(goods, brands)

0.6101096747940727